## Aim

This code does the following: 
- Extracts Indian data from the [Planted Trees map](https://www.globalforestwatch.org/blog/data-and-tools/updated-planted-trees-map-near-global-coverage/) from the World Resources Institute (WRI). From this link, you can also download the data.
- Converts the GeoPackage data into a shape file (such that it can be ingested in Google Earth Engine (GEE))

More information on this dataset can also be found [here](https://www.wri.org/research/spatial-database-planted-trees-sdpt-version-2?ap3c=IGkMIRz9Ic8arE0EAGkMIRxggQ62SFTnOwQXKFOP5omGChdpJA).


In this notebook, GDAL is used to extract different shapefiles (.shp extension) and this for separate years. The following command in GDAL was used:

```bash
ogr2ogr -f "ESRI Shapefile" india_plant_v2.shp india_plant_v2.gpkg ind_plant_v2
```

The command line gave several warnings including one where  "2GB file size limit reached for india_plant_v2.dbf". Hence, some clean up will be done here and some variables dropped such that one file can be uploaded in Google Earth Engine (GEE). 

In order to install GDAL using Python, an excellent resource is the course material of Ujaval Gandhi ["Mastering GDAL Tools (Full Course)"](https://courses.spatialthoughts.com/).

## Reading in the Python packages

In [15]:
import glob
import re
import subprocess
import os

import pandas as pd

In [16]:
## input_dir = "data" # I am running the program in the same directory
input_dir_complete = r"D:\sdpt_v2_v20231128.gpkg"# Path for the entire big file - External hard disk in my case
gpkg_india = 'india_plant_v2.gpkg'

## Extracting India data

First, get some information of the entire file:

In [17]:
cmd = ["ogrinfo", input_dir_complete]

result = subprocess.run(cmd, capture_output=True, text=True)
## print(result.stdout) ## Uncomment this line to see the results. This shows multi polygons per country 


Extract India:

In [18]:
import subprocess

cmd = [
    "ogr2ogr",
    "-f", "GPKG",
    gpkg_india,
    input_dir_complete,
    "ind_plant_v2"
]

result = subprocess.run(cmd, capture_output=True, text=True)

print(result.stdout)
print(result.stderr)  # shows warnings/errors


## Converting the geopackage to shape files
First, just check the newly created India file:

In [19]:
cmd = [
        "ogrinfo",
        "-so", "-al",
         gpkg_india
        ]
result = subprocess.run(cmd, check=True, capture_output=True, text=True)

if result.returncode == 0:
    print("ogrinfo ran successfully!\n")
    # Split output into lines and format it
    lines = [line.strip() for line in result.stdout.splitlines() if line.strip()]
    for line in lines:
        print(line)
else:
    print("Error running ogrinfo:")
    print(result.stderr)

ogrinfo ran successfully!

INFO: Open of `india_plant_v2.gpkg'
using driver `GPKG' successful.
Layer name: ind_plant_v2
Geometry: Multi Polygon
Feature Count: 1247437
Extent: (69.021570, 6.752422) - (97.180654, 34.820436)
Layer SRS WKT:
GEOGCRS["WGS 84",
ENSEMBLE["World Geodetic System 1984 ensemble",
MEMBER["World Geodetic System 1984 (Transit)"],
MEMBER["World Geodetic System 1984 (G730)"],
MEMBER["World Geodetic System 1984 (G873)"],
MEMBER["World Geodetic System 1984 (G1150)"],
MEMBER["World Geodetic System 1984 (G1674)"],
MEMBER["World Geodetic System 1984 (G1762)"],
MEMBER["World Geodetic System 1984 (G2139)"],
MEMBER["World Geodetic System 1984 (G2296)"],
ELLIPSOID["WGS 84",6378137,298.257223563,
LENGTHUNIT["metre",1]],
ENSEMBLEACCURACY[2.0]],
PRIMEM["Greenwich",0,
ANGLEUNIT["degree",0.0174532925199433]],
CS[ellipsoidal,2],
AXIS["geodetic latitude (Lat)",north,
ORDER[1],
ANGLEUNIT["degree",0.0174532925199433]],
AXIS["geodetic longitude (Lon)",east,
ORDER[2],
ANGLEUNIT["degree",0

Check also the coordinate system as GEE needs epsg 4326:

In [20]:
cmd = [
    "gdalsrsinfo",
    "-o", "epsg",
    gpkg_india  
]

# Run the command and capture the output
result = subprocess.run(cmd, capture_output=True, text=True)

# Print the EPSG code
print("EPSG code:", result.stdout.strip())

EPSG code: EPSG:4326


Check the different planted years and their count and get the output into a list to get an idea of the distribution across the years

In [11]:
import subprocess
import re
layer_name = "ind_plant_v2"

cmd = [
    "ogrinfo",
    gpkg_india,
    "-sql",
    f'SELECT plantedYear, COUNT(*) AS count FROM {layer_name} GROUP BY plantedYear ORDER BY plantedYear'
]

# Run the command
result = subprocess.run(cmd, capture_output=True, text=True)

# Raw output
output = result.stdout

# Optional: print warnings/errors
if result.stderr:
    print("Warnings / Errors:\n", result.stderr)

In [12]:
planted_years = []

print("PlantedYear counts:")
for feat in features:
    if not feat.strip():
        continue
    year_match = re.search(r'plantedYear.*= (\d+)', feat)
    count_match = re.search(r'count.*= (\d+)', feat)
    if year_match and count_match:
        year = int(year_match.group(1))
        count = int(count_match.group(1))
        print(f"{year}: {count}")
        planted_years.append(year)

# Now planted_years contains all unique years
print("List of planted years:", planted_years)

PlantedYear counts:
1982: 184032
1983: 1
1986: 1
1987: 17
1988: 7990
1989: 13976
1990: 15701
1991: 5257
1992: 66324
1993: 14831
1994: 128666
1995: 3918
1996: 22741
1997: 19858
1998: 18231
1999: 10988
2000: 8179
2001: 7883
2002: 35753
2003: 3652
2004: 19302
2005: 47721
2006: 9905
2007: 18733
2008: 9917
2009: 27190
2010: 51044
2011: 6666
2012: 93325
2013: 10676
2014: 14050
2015: 12441
2016: 8978
2017: 10393
2018: 13776
2019: 11666
List of planted years: [1982, 1983, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]


Do the same but this for the different plantations:

In [13]:
cmd = [
    "ogrinfo",
    gpkg_india,
    "-sql",
    f'SELECT originalName, COUNT(*) AS count FROM {layer_name} GROUP BY originalName ORDER BY originalName'
]

result = subprocess.run(cmd, capture_output=True, text=True)
output = result.stdout

if result.stderr:
    print("Warnings / Errors:\n", result.stderr)

features = re.split(r'OGRFeature\(.+\)', output)

print("OriginalName counts:")

for feat in features:
    if not feat.strip():
        continue
    name_match = re.search(r'originalNa.*= (.+)', feat)
    count_match = re.search(r'count.*= (\d+)', feat)
    if name_match and count_match:
        name = name_match.group(1).strip()
        count = count_match.group(1)
        print(f"{name}: {count}")


OriginalName counts:
Acacia: 3121
Almonds: 740
Alnus: 19
Apples: 151
Areca: 12853
Cashew nut: 62
Casuriana: 335
Citrus: 11
Coconut: 1023
Coffee: 10
Cryptomeria: 146
Eucalyptus: 12131
Gliricidia plantation: 153
Mango: 3002
Mixed plantation: 55795
Oil Palm: 8859
Oil palm: 89
Orchard: 298261
Padauk: 109
Pine: 667078
Red oil palm: 36
Rubber: 157
Sal: 116489
Tea: 523
Teak: 66284


If we want to convert the entire GeoPackage to a shape file, we get some warnings (which may potentially read to problems). This is what the command gives. Please uncomment to see the output (and especially the warnings at the last line where the .dbf file is large):

In [21]:
cmd = [
    "ogr2ogr",
    "-f", "ESRI Shapefile",
    "india_plant_v2.shp",
    "india_plant_v2.gpkg",
    "ind_plant_v2"
]

print("Running GDAL command:\n", " ".join(cmd), "\n")

# Run command and stream GDAL output live to the console
process = subprocess.Popen(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)

for line in process.stdout:
    print(line, end="")  # Print each line as it comes

process.wait()

if process.returncode == 0:
    print("\nConversion completed successfully!")
else:
    print(f"\nogr2ogr failed with return code {process.returncode}")


Running GDAL command:
 ogr2ogr -f ESRI Shapefile india_plant_v2.shp india_plant_v2.gpkg ind_plant_v2 

Warning 1: Field final_id of width 255 truncated to 254.
Warning 1: Field iso3 of width 8000 truncated to 254.
Warning 1: Field iso2 of width 8000 truncated to 254.
Warning 1: Field country of width 8000 truncated to 254.
Warning 6: Normalized/laundered field name: 'originalName' to 'originalNa'
Warning 1: Field originalNa of width 8000 truncated to 254.
Warning 6: Normalized/laundered field name: 'originalCode' to 'originalCo'
Warning 1: Field vernacName of width 8000 truncated to 254.
Warning 1: Field sciName of width 8000 truncated to 254.
Warning 1: Field simpleName of width 8000 truncated to 254.
Warning 1: Field sciName1 of width 8000 truncated to 254.
Warning 1: Field sciName2 of width 8000 truncated to 254.
Warning 1: Field sciName3 of width 8000 truncated to 254.
Warning 1: Field sciName4 of width 8000 truncated to 254.
Warning 1: Field sciName5 of width 8000 truncated to 254

We can drop some variables and only retain the originalName and the plantedYear:

In [28]:
cmd = ["ogrinfo", "india_plant_v2.gpkg", "-so", "-al"]

result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout) ## Uncomment this line to see the results. This shows multi polygons per country 


INFO: Open of `india_plant_v2.gpkg'
      using driver `GPKG' successful.

Layer name: ind_plant_v2
Geometry: Multi Polygon
Feature Count: 1247437
Extent: (69.021570, 6.752422) - (97.180654, 34.820436)
Layer SRS WKT:
GEOGCRS["WGS 84",
    ENSEMBLE["World Geodetic System 1984 ensemble",
        MEMBER["World Geodetic System 1984 (Transit)"],
        MEMBER["World Geodetic System 1984 (G730)"],
        MEMBER["World Geodetic System 1984 (G873)"],
        MEMBER["World Geodetic System 1984 (G1150)"],
        MEMBER["World Geodetic System 1984 (G1674)"],
        MEMBER["World Geodetic System 1984 (G1762)"],
        MEMBER["World Geodetic System 1984 (G2139)"],
        MEMBER["World Geodetic System 1984 (G2296)"],
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]],
        ENSEMBLEACCURACY[2.0]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1

In [49]:
# Writing directly to shp file, I had an error, so dropping variables first
input_gpkg = "india_plant_v2.gpkg"
subset_gpkg = "india_plant_v2_subset.gpkg"
layer_name = "ind_plant_v2"

sql = f"SELECT plantedYear, originalName, Shape FROM {layer_name}"

cmd = [
    "ogr2ogr",
    "-overwrite",
    "-f", "GPKG",
    subset_gpkg,
    input_gpkg,
    "-sql", sql,
    "-nln", "ind_plant_subset" 
]

# Run the command
result = subprocess.run(cmd, capture_output=True, text=True)

print(result.stdout)
if result.stderr:
    print("Warnings / Errors:\n", result.stderr)

In [50]:
cmd = ["ogrinfo", subset_gpkg, "-so", "-al"]

result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout) ## Uncomment this line to see the results. This shows multi polygons per country 

INFO: Open of `india_plant_v2_subset.gpkg'
      using driver `GPKG' successful.

Layer name: ind_plant_subset
Geometry: Multi Polygon
Feature Count: 1247437
Extent: (69.021570, 6.752422) - (97.180654, 34.820436)
Layer SRS WKT:
GEOGCRS["WGS 84",
    ENSEMBLE["World Geodetic System 1984 ensemble",
        MEMBER["World Geodetic System 1984 (Transit)"],
        MEMBER["World Geodetic System 1984 (G730)"],
        MEMBER["World Geodetic System 1984 (G873)"],
        MEMBER["World Geodetic System 1984 (G1150)"],
        MEMBER["World Geodetic System 1984 (G1674)"],
        MEMBER["World Geodetic System 1984 (G1762)"],
        MEMBER["World Geodetic System 1984 (G2139)"],
        MEMBER["World Geodetic System 1984 (G2296)"],
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]],
        ENSEMBLEACCURACY[2.0]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
        

In [58]:
# Convert now to a shape file:
cmd = [
    "ogr2ogr",
    "-f", "ESRI Shapefile",
    "india_plant_v2_subset.shp",
    "india_plant_v2_subset.gpkg",
    "ind_plant_subset",
    "-a_srs", "EPSG:4326"  # assign projection
]

print("Running GDAL command:\n", " ".join(cmd), "\n")

# Run command and stream GDAL output live to the console
process = subprocess.Popen(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)

for line in process.stdout:
    print(line, end="")  # Print each line as it comes

process.wait()

if process.returncode == 0:
    print("\nConversion completed successfully!")
else:
    print(f"\nogr2ogr failed with return code {process.returncode}")


Running GDAL command:
 ogr2ogr -f ESRI Shapefile india_plant_v2_subset.shp india_plant_v2_subset.gpkg ind_plant_subset -a_srs EPSG:4326 

Warning 6: Normalized/laundered field name: 'plantedYear' to 'plantedYea'
Warning 6: Normalized/laundered field name: 'originalName' to 'originalNa'
Warning 1: Field originalNa of width 8000 truncated to 254.

Conversion completed successfully!


In [61]:
base = "india_plant_v2_subset"  

## Collect all parts of the shape file
files = glob.glob(base + ".*")
print(files)

# Only keep those files that have the dbf, prj, shp or shx extension:
allowed_exts = {".shp", ".prj", ".dbf", ".shx"}
filtered = [f for f in files if os.path.splitext(f)[1].lower() in allowed_exts]

total_bytes = sum(os.path.getsize(f) for f in filtered)
total_gb = total_bytes / (1024 ** 3)

print("Shapefile components:")
for f in filtered:
    size_mb = os.path.getsize(f) / (1024 ** 2)
    print(f"  {os.path.basename(f)}: {size_mb:.2f} MB")

print(f"\nTotal size: {total_gb:.3f} GB")

['india_plant_v2_subset.dbf', 'india_plant_v2_subset.gpkg', 'india_plant_v2_subset.prj', 'india_plant_v2_subset.shp', 'india_plant_v2_subset.shx']
Shapefile components:
  india_plant_v2_subset.dbf: 314.07 MB
  india_plant_v2_subset.prj: 0.00 MB
  india_plant_v2_subset.shp: 567.49 MB
  india_plant_v2_subset.shx: 9.52 MB

Total size: 0.870 GB


In [62]:
cmd = [
    "gdalsrsinfo",
    "-o", "epsg",
    "india_plant_v2_subset.shp"   
]

# Run the command and capture the output
result = subprocess.run(cmd, capture_output=True, text=True)

# Print the EPSG code
print("EPSG code:", result.stdout.strip())

EPSG code: EPSG:4326


Now, the file can be exported to Google Earth Engine. You can upload the different components or create a zip file.

In [64]:
cmd = ["ogrinfo", "india_plant_v2_subset.shp", "-so", "-al"]

result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout) ## Uncomment this line to see the results. This shows multi polygons per country 


INFO: Open of `india_plant_v2_subset.shp'
      using driver `ESRI Shapefile' successful.

Layer name: india_plant_v2_subset
Metadata:
  DBF_DATE_LAST_UPDATE=2025-11-13
Geometry: Polygon
Feature Count: 1247437
Extent: (69.021570, 6.752422) - (97.180654, 34.820436)
Layer SRS WKT:
GEOGCRS["WGS 84",
    DATUM["World Geodetic System 1984",
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["latitude",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["longitude",east,
            ORDER[2],
            ANGLEUNIT["degree",0.0174532925199433]],
    ID["EPSG",4326]]
Data axis to CRS axis mapping: 2,1
plantedYea: Integer (9.0)
originalNa: String (254.0)

